# 参数初始化

本文参考邱锡鹏[《神经网络与深度学习》](https://nndl.github.io)、[知乎文章](https://zhuanlan.zhihu.com/p/39015193)、[知乎文章](https://zhuanlan.zhihu.com/p/57454669)

神经网络的参数学习是一个非凸优化问题. 当使用梯度下降法来进行优化 网络参数时，参数初始值的选取十分关键，关系到网络的优化效率和泛化能力. 参数初始化的方式通常有以下三种
1. 预训练初始化
  通常情 况下，一个已经在大规模数据上训练过的模型可以提供一个好的参数初始 值，这种初始化方法称为预训练初始化(Pretrained Initialization)，预训练模型在目标任务上学习的过程也被称为Fine-tuning.

2. 随机初始化
  在线性模型的训练(比如Logistic回归)中，我们一般将参数全部初始化为0。但是这在神经网络的训练中会存在一些问题. 因为如果参数都为0，考虑公式$\delta^l = f_l'(z^l)\odot W^{l + 1}\delta^{l + 1}$，所有的$\delta^l$ 值都为零，那么反向传播算法那在这个时候回失效。为了不出现这种情况，较好的方式就是选择对每个参数都随机初始化.

3. 固定值初始化
  对于一些特殊的参数，我们可以根据经验用一个特殊的固定的值来初始化，比如偏置(Bias)通常用0来初始化。在LSTM网络的遗忘门中，偏置通常初始为1或2，使得时序上的梯度变大. 对于使用ReLU的神经元，有时也可以将偏置设为0.01，使得ReLU神经元在训练初期更容易激活，从而获得一定的梯度来进行误差反向传播.

## 基于固定方差的参数初始化

1. Normal Distribution
  大家在学习高斯分布的时候应该了解过它的另一个名字，正态分布，它表明自然界中的绝大多数变量都是服从高斯分布的。所以，很直观的一个想法就是使用高斯分布来初始化参数。通常是这个均值为0,方差为$\sigma^2$
2. Uniform Distribution
  使用均匀分布初始化，在给定的一个区间$[a, b]$内采用均匀分布来初始化参数其方差为
  $$\text{var}(x) = \frac{(b - a)^2}{12}\tag{1}$$

这两种方法最关键的就是设置方差$\sigma^2$，参数范围太小会导致神经网络的输出过小，经过多层之后信号就慢慢消失了，参数范围太大对于Sigmoid或者Tanh就会出现梯度消失的问题。

## 基于方差缩放的参数初始化

初始化一个深度网络时，为了缓解梯度消失或爆炸问题，我们尽可能保持每 个神经元的输入和输出的方差一致，根据神经元的连接数量来自适应地调整初 始化分布的方差，这类方法称为方差缩放(Variance Scaling)

这种初始化的方法最基本的思想就是尽可能地保持神经元的输入和输出的方差一致

假设地$l$层的神经元为$M_l$个, $l - 1$层的神经元为$M_{l - 1}$且其输出为$a^{(l - 1)}$, 先假设网络层的激活函数$f(x) = x$。

并且假设$w_i^{(l)}$和$a_i^{(l - 1)}$, $w$和$a$分别独立同分布，则$a^{(l)}$的均值为
$$
E(a^{(l)}) = E\left(\displaystyle\sum_{i = 1}^{M_{l - 1}}w_i^{(l)}a_i^{(l - 1)}\right) = \sum_{i = 1}^{M_{l - 1}}E(w_i^{(l)})E(a_i^{(l - 1)}) = 0\tag{2}
$$
$a_{(l)}$的方差为
$$
\begin{aligned}\text{var}(a^{(l)})=& \text{var}\left(\displaystyle\sum_{i = 1}^{M_{l - 1}}w_i^{(l)}a_i^{(l - 1)}\right)\\=& \sum_{i = 1}^{M_{l - 1}}\text{var}(w_i^{(l)})\text{var}(a_i^{(l - 1)})\\=& M_{l - 1}\text{var}(w_i^{(l)})\text{var}(a_i^{(l - 1)})\end{aligned}\tag{3}
$$


那么，为了前向计算的时候输入和输出端的方差一致,则设置
$$
\text{var}(w_i^{(l)}) = \frac{1}{M_{l - 1}}\tag{4}
$$
同理，反向传播的时候，应该设置
$$
\text{var}(w_i^{(l)}) = \frac{1}{M_{l}}\tag{5}
$$
折中之后，设置
$$
\text{var}(w_i^{(l)}) = \frac{2}{M_{l - 1} + M_l}\tag{5}
$$
因为神经元的参数和输入的绝对值通常比较小，处于激活函数的线性区间. 这时Sigmoid函数和Tanh函数可以近似为 线性函数.  Logistic 函数在线性区间的斜率约为0.25, Tanh约为1。使用ReLU作为激活函数式，通常只有一半的神经元有效。

![Variance Scaling](imgs/para_init.png)